In [18]:
import pretty_midi as pm
from mido import MidiFile
import music21
import os
from glob import glob
import pandas as pd
import re
import copy

In [142]:
folder = "./data/Complete Examples"

files = [y for x in os.walk(folder) for y in glob(os.path.join(x[0], '*.mid'))]

In [8]:
# # TEMP
# files = ["./data/Complete Examples/Jazz-Midi/All Of Me.mid"]

In [9]:
melody_names = ['melody', 'melodia', 'melodía', 'lead']
non_melody_names = ['bass', 'bajo', 'basso', 'baixo', 'drum', 'percussion', 'batería', 'bateria', 'chord', 'rhythm',
                     'cymbal', 'clap', 'kick', 'snare', 'hh ', 'hats', 'ride', 'kit']

In [10]:
def get_channels(m):
    channels = set({})
    for t in m.tracks:
        for m in t:
            if m.type == 'note_on':
                channels.add(m.channel)
                    
    return channels

In [11]:
def get_programs_mido(m, channels):
    programs = []
    
    for c in channels:
        programs.append(tuple([0, c]))
    
    for t in m.tracks:
        for m in t:
            if m.type == 'program_change':
                programs.append(tuple([m.program, m.channel]))
                
    return programs

In [12]:
def get_programs_pm(m):
    programs = []
    
    for i in m.instruments:
        programs.append(tuple([i.program, i.is_drum]))
        
    return programs

In [13]:
def filter_instruments(pm_p):
    filtered_instr = [
        i for i in pm_p.instruments 
        if i.is_drum == False
        and all(
            sub not in i.name.lower() for sub in non_melody_names
        )]
    
    return filtered_instr

In [14]:
def get_melody_tracks(pm_p):
    melody_tracks = [i for i in pm_p.instruments if 'solo' in i.name.lower()]
    
    if len(melody_tracks) == 0:
        melody_tracks = [
            i for i in pm_p.instruments
            if any(
                sub in i.name.lower() for sub in melody_names
            )]
    
    return melody_tracks

In [23]:
def extract(file):
    status = 'unprocessed'
    error = None
    
    source = os.path.basename(os.path.dirname(file))
    
    single_melody = False
    multiple_melody = False
    solo_channel = False
    multi_channel = False
    disagreement = False
    single_candidate = False
    
    melody_tracks = []
    filtered_instr = []
    melody_track = None
    
    song_name = os.path.basename(file).replace('.* - ', '').replace('.mid', '')
    song_name = re.sub(r'\([0-9]*\)', '', song_name)

    melody_track_name = None
    
    channels = None
    candidate_names = []
    n_channels = None
    n_instr = None
    mido_p = None
    pm_m = None
    pm_p = None
    midi_type = None
    
    instruments = []
    
    try:
        mido_m = MidiFile(file, clip=True)
        midi_type = mido_m.type

        pm_m = pm.PrettyMIDI(file)

        channels = get_channels(mido_m)
        mido_p = get_programs_mido(mido_m, channels)
        pm_p = get_programs_pm(pm_m)
        
        instruments = pm_m.instruments

        for p, _ in pm_p:
            if p not in [m[0] for m in mido_p]:
                error = 'program mismatch'

        melody_tracks = get_melody_tracks(pm_m)

        n_tracks = len(mido_m.tracks)
        n_channels = len(channels)
        n_instr = len(pm_m.instruments)

        filtered_instr = filter_instruments(pm_m)
        candidate_names = [i.name for i in filtered_instr]

        if n_channels == 1 & n_tracks == 1:
            solo_channel = True
            melody_track = pm_m.instruments[0]    

        if len(melody_tracks) == 1:
            melody_track = melody_tracks[0]
            single_melody = True
            single_candidate = True

        if len(melody_tracks) > 1:
            multiple_melody = True

        if len(filtered_instr) == 1:
            melody_track = filtered_instr[0]
            single_candidate = True

        if len(filtered_instr) > 1:
            multi_channel = True

        if n_channels != n_instr:
            disagreement = True


        if single_melody or solo_channel or single_candidate:
            status = 'processed'
            
            out_path = os.path.join('.', 'data', 'Complete Examples Melodies')
            
           
        if status == 'unprocessed':
            
            out_path = os.path.join('.', 'data', 'Complete Examples Melodies Random')
            
            melody_track  = filtered_instr[0]
           
        
        if melody_track:
            melody_track_name = melody_track.name
            
        if not os.path.exists(os.path.join(out_path, source)):
            os.mkdir(os.path.join(out_path, source))
                
        out_filename = os.path.join(out_path, source, os.path.basename(file))

        pm_melody = copy.deepcopy(pm_m)
        pm_melody.instruments = [melody_track]
        pm_melody.write(out_filename) 
            
        
    except Exception as e:
        print(f, e)
        error = e
    
    
    return {
       'filename': file,
       'source': source,
       'song_name': song_name,
       'midi_type' : midi_type,
       'channels': channels,
       'n_channels': n_channels,
       'instruments': instruments,
       'n_instruments': n_instr,
       'programs_mido': mido_p,
       'programs_pm': pm_p,
       'melody_tracks': melody_tracks,
       'melody_track': melody_track,
       'melody_track_name': melody_track_name,
       'candidates': filtered_instr,
       'candidate_names': candidate_names,
       'single_melody': single_melody,
       'multiple_melody': multiple_melody,
       'solo_channel': solo_channel, 
       'single_candidate': single_candidate,
       'multi_channel': multi_channel,
       'disagreement': disagreement,
       'status': status,
       'error': error
   }

In [24]:
len(files)

1

In [25]:
extract(files[0])

{'filename': './data/Complete Examples/Jazz-Midi/All Of Me.mid',
 'source': 'Jazz-Midi',
 'song_name': 'All Of Me',
 'midi_type': 1,
 'channels': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10},
 'n_channels': 11,
 'instruments': [Instrument(program=0, is_drum=True, name="-<>-<>-<>-"),
  Instrument(program=0, is_drum=True, name="\"All of Me\""),
  Instrument(program=0, is_drum=True, name="Transcribed"),
  Instrument(program=0, is_drum=True, name="& Digitized"),
  Instrument(program=0, is_drum=True, name="For MIDI By"),
  Instrument(program=0, is_drum=True, name="GaryW0001"),
  Instrument(program=0, is_drum=True, name="Music By:"),
  Instrument(program=0, is_drum=True, name="S. Simons &"),
  Instrument(program=0, is_drum=True, name="Gerry Marks"),
  Instrument(program=0, is_drum=True, name="Arranged By"),
  Instrument(program=0, is_drum=True, name="Billy Byers"),
  Instrument(program=0, is_drum=True, name="Tempo=130"),
  Instrument(program=0, is_drum=True, name="Comments?"),
  Instrument(program=0, 

In [26]:
songs_list = []
for f in files:
    song = extract(f)
    songs_list.append(song)

In [27]:
columns = ['filename', 'source', 'song_name', 'midi_type', 'channels', 
           'n_channels', 'instruments', 'n_instruments', 
           'programs_mido', 'programs_pm', 'melody_tracks', 'melody_track', 
           'melody_track_name', 'candidates', 'candidate_names', 'single_melody', 'multiple_melody', 
           'solo_channel', 'single_candidate', 'multi_channel', 'disagreement', 'status', 'error']
songs = pd.DataFrame(
   songs_list, 
   columns=columns
)

In [28]:
output_csv = './data/songs/v1.csv'

if not os.path.exists(output_csv):
    songs.to_csv(output_csv)
else:
    print('File already exists!')

In [155]:
songs

,filename,source,song_name,midi_type,channels,n_channels,instruments,n_instruments,programs_mido,programs_pm,...,candidates,candidate_names,single_melody,multiple_melody,solo_channel,single_candidate,multi_channel,disagreement,status,error
0,./data/Complete Examples/Doug McKenzie/After Y...,Doug McKenzie,After You've Gone,1.0,"{8, 1, 9, 7}",4.0,"[Instrument(program=0, is_drum=False, name=""Tr...",4.0,"[(0, 8), (0, 1), (0, 9), (0, 7), (35, 1), (32,...","[(0, False), (0, False), (35, False), (32, True)]",...,"[Instrument(program=0, is_drum=False, name=""Tr...","[Track 2, Track 2, Track 3]",False,False,False,False,True,False,unprocessed,None
1,./data/Complete Examples/Doug McKenzie/Alice I...,Doug McKenzie,Alice In Wonderland,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""do...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""do...",[dougmck@pipeline.com.au],False,False,False,True,False,False,processed,None
2,./data/Complete Examples/Doug McKenzie/All The...,Doug McKenzie,All The Things You Are,1.0,"{0, 1, 9}",3.0,"[Instrument(program=0, is_drum=False, name="""")...",3.0,"[(0, 0), (0, 1), (0, 9), (0, 0), (83, 3), (32,...","[(0, False), (32, False), (32, True)]",...,"[Instrument(program=0, is_drum=False, name="""")...","[, ]",False,False,False,False,True,False,unprocessed,None
3,./data/Complete Examples/Doug McKenzie/All The...,Doug McKenzie,All The Things You Are,1.0,"{0, 1, 9}",3.0,"[Instrument(program=0, is_drum=False, name="""")...",3.0,"[(0, 0), (0, 1), (0, 9), (0, 0), (83, 3), (32,...","[(0, False), (32, False), (32, True)]",...,"[Instrument(program=0, is_drum=False, name="""")...","[, ]",False,False,False,False,True,False,unprocessed,None
4,./data/Complete Examples/Doug McKenzie/All The...,Doug McKenzie,All The Things You Are,1.0,"{0, 1, 2, 9}",4.0,"[Instrument(program=32, is_drum=False, name=""""...",4.0,"[(0, 0), (0, 1), (0, 2), (0, 9), (0, 0), (32, ...","[(32, False), (0, True), (0, False), (35, False)]",...,"[Instrument(program=32, is_drum=False, name=""""...","[, , ]",False,False,False,False,True,False,unprocessed,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,./data/Complete Examples/Weimar DB/Woody Shaw ...,Weimar DB,Woody Shaw - Imagination,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Me...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Me...",[Melody],True,False,True,True,False,False,processed,None
661,./data/Complete Examples/Weimar DB/Wynton Mars...,Weimar DB,Wynton Marsalis - April In Paris,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name="""")]",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name="""")]",[],False,False,True,True,False,False,processed,None
662,./data/Complete Examples/Weimar DB/Zoot Sims -...,Weimar DB,Zoot Sims - All The Things You Are,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Me...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Me...",[Melody],True,False,True,True,False,False,processed,None
663,./data/Complete Examples/Weimar DB/Zoot Sims -...,Weimar DB,Zoot Sims - Night And Day,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Me...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Me...",[Melody],True,False,True,True,False,False,processed,None


In [156]:
songs[songs['error'] == 'program mismatch'][['programs_mido', 'programs_pm']]

,programs_mido,programs_pm


In [157]:
songs[~songs['error'].isnull()]['error'].value_counts()

data byte must be in range 0..127                 1
data byte must be in range 0..127                 1
                                                  1
                                                  1
Could not decode key with 6 flats and mode 255    1
Name: error, dtype: int64

In [158]:
songs[~songs.melody_track.isnull()]

,filename,source,song_name,midi_type,channels,n_channels,instruments,n_instruments,programs_mido,programs_pm,...,candidates,candidate_names,single_melody,multiple_melody,solo_channel,single_candidate,multi_channel,disagreement,status,error
0,./data/Complete Examples/Doug McKenzie/After Y...,Doug McKenzie,After You've Gone,1.0,"{8, 1, 9, 7}",4.0,"[Instrument(program=0, is_drum=False, name=""Tr...",4.0,"[(0, 8), (0, 1), (0, 9), (0, 7), (35, 1), (32,...","[(0, False), (0, False), (35, False), (32, True)]",...,"[Instrument(program=0, is_drum=False, name=""Tr...","[Track 2, Track 2, Track 3]",False,False,False,False,True,False,unprocessed,None
1,./data/Complete Examples/Doug McKenzie/Alice I...,Doug McKenzie,Alice In Wonderland,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""do...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""do...",[dougmck@pipeline.com.au],False,False,False,True,False,False,processed,None
2,./data/Complete Examples/Doug McKenzie/All The...,Doug McKenzie,All The Things You Are,1.0,"{0, 1, 9}",3.0,"[Instrument(program=0, is_drum=False, name="""")...",3.0,"[(0, 0), (0, 1), (0, 9), (0, 0), (83, 3), (32,...","[(0, False), (32, False), (32, True)]",...,"[Instrument(program=0, is_drum=False, name="""")...","[, ]",False,False,False,False,True,False,unprocessed,None
3,./data/Complete Examples/Doug McKenzie/All The...,Doug McKenzie,All The Things You Are,1.0,"{0, 1, 9}",3.0,"[Instrument(program=0, is_drum=False, name="""")...",3.0,"[(0, 0), (0, 1), (0, 9), (0, 0), (83, 3), (32,...","[(0, False), (32, False), (32, True)]",...,"[Instrument(program=0, is_drum=False, name="""")...","[, ]",False,False,False,False,True,False,unprocessed,None
4,./data/Complete Examples/Doug McKenzie/All The...,Doug McKenzie,All The Things You Are,1.0,"{0, 1, 2, 9}",4.0,"[Instrument(program=32, is_drum=False, name=""""...",4.0,"[(0, 0), (0, 1), (0, 2), (0, 9), (0, 0), (32, ...","[(32, False), (0, True), (0, False), (35, False)]",...,"[Instrument(program=32, is_drum=False, name=""""...","[, , ]",False,False,False,False,True,False,unprocessed,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,./data/Complete Examples/Weimar DB/Woody Shaw ...,Weimar DB,Woody Shaw - Imagination,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Me...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Me...",[Melody],True,False,True,True,False,False,processed,None
661,./data/Complete Examples/Weimar DB/Wynton Mars...,Weimar DB,Wynton Marsalis - April In Paris,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name="""")]",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name="""")]",[],False,False,True,True,False,False,processed,None
662,./data/Complete Examples/Weimar DB/Zoot Sims -...,Weimar DB,Zoot Sims - All The Things You Are,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Me...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Me...",[Melody],True,False,True,True,False,False,processed,None
663,./data/Complete Examples/Weimar DB/Zoot Sims -...,Weimar DB,Zoot Sims - Night And Day,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Me...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Me...",[Melody],True,False,True,True,False,False,processed,None


In [159]:
songs[songs.melody_track.isnull()]

,filename,source,song_name,midi_type,channels,n_channels,instruments,n_instruments,programs_mido,programs_pm,...,candidates,candidate_names,single_melody,multiple_melody,solo_channel,single_candidate,multi_channel,disagreement,status,error
92,./data/Complete Examples/Jazz-Midi/Darn That D...,Jazz-Midi,Darn That Dream,1.0,None,NaN,[],NaN,None,None,...,[],[],False,False,False,False,False,False,unprocessed,data byte must be in range 0..127
280,./data/Complete Examples/MidKar/Maiden Voyage.mid,MidKar,Maiden Voyage,1.0,None,NaN,[],NaN,None,None,...,[],[],False,False,False,False,False,False,unprocessed,data byte must be in range 0..127
319,./data/Complete Examples/Oocities/Another You.mid,Oocities,Another You,NaN,None,NaN,[],NaN,None,None,...,[],[],False,False,False,False,False,False,unprocessed,
324,./data/Complete Examples/Oocities/Footprints.mid,Oocities,Footprints,NaN,None,NaN,[],NaN,None,None,...,[],[],False,False,False,False,False,False,unprocessed,
343,./data/Complete Examples/Oocities/Round Midnig...,Oocities,Round Midnight,NaN,None,NaN,[],NaN,None,None,...,[],[],False,False,False,False,False,False,unprocessed,Could not decode key with 6 flats and mode 255


In [160]:
songs[songs.multiple_melody].shape

(21, 23)

In [161]:
songs[(songs.single_melody | songs.solo_channel)]

,filename,source,song_name,midi_type,channels,n_channels,instruments,n_instruments,programs_mido,programs_pm,...,candidates,candidate_names,single_melody,multiple_melody,solo_channel,single_candidate,multi_channel,disagreement,status,error
9,./data/Complete Examples/Doug McKenzie/But Bea...,Doug McKenzie,But Beautiful,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""So...",1.0,"[(0, 0), (0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""So...",[Solo piano],True,False,False,True,False,False,processed,None
10,./data/Complete Examples/Doug McKenzie/Come Ra...,Doug McKenzie,Come Rain Or Come Shine,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Ad...",2.0,"[(0, 0)]","[(0, False), (0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Ad...","[Added solo track, Accompaniment]",True,False,False,True,True,True,processed,None
13,./data/Complete Examples/Doug McKenzie/Cry Me ...,Doug McKenzie,Cry Me A River,0.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""un...",1.0,"[(0, 0), (0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""un...",[untitled],False,False,True,True,False,False,processed,None
15,./data/Complete Examples/Doug McKenzie/Estate.mid,Doug McKenzie,Estate,0.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Es...",1.0,"[(0, 0), (0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Es...",[Estate],False,False,True,True,False,False,processed,None
20,./data/Complete Examples/Doug McKenzie/Have Yo...,Doug McKenzie,Have You Met Miss Jones,1.0,"{0, 1}",2.0,"[Instrument(program=0, is_drum=False, name=""So...",2.0,"[(0, 0), (0, 1), (0, 1)]","[(0, False), (0, False)]",...,"[Instrument(program=0, is_drum=False, name=""So...","[Solo over accompaniment , Accompaniment]",True,False,False,True,True,False,processed,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,./data/Complete Examples/Weimar DB/Woody Shaw ...,Weimar DB,Woody Shaw - Imagination,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Me...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Me...",[Melody],True,False,True,True,False,False,processed,None
661,./data/Complete Examples/Weimar DB/Wynton Mars...,Weimar DB,Wynton Marsalis - April In Paris,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name="""")]",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name="""")]",[],False,False,True,True,False,False,processed,None
662,./data/Complete Examples/Weimar DB/Zoot Sims -...,Weimar DB,Zoot Sims - All The Things You Are,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Me...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Me...",[Melody],True,False,True,True,False,False,processed,None
663,./data/Complete Examples/Weimar DB/Zoot Sims -...,Weimar DB,Zoot Sims - Night And Day,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Me...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Me...",[Melody],True,False,True,True,False,False,processed,None


In [162]:
songs.shape

(665, 23)

In [163]:
songs.single_melody.value_counts()

True     372
False    293
Name: single_melody, dtype: int64

In [164]:
songs.multiple_melody.value_counts()

False    644
True      21
Name: multiple_melody, dtype: int64

In [165]:
songs.solo_channel.value_counts()

False    536
True     129
Name: solo_channel, dtype: int64

In [166]:
songs.multi_channel.value_counts()

True     473
False    192
Name: multi_channel, dtype: int64

In [167]:
songs.single_candidate.value_counts()

True     456
False    209
Name: single_candidate, dtype: int64

In [168]:
songs.disagreement.value_counts()

False    579
True      86
Name: disagreement, dtype: int64

In [169]:
songs.error.value_counts()

data byte must be in range 0..127                 1
data byte must be in range 0..127                 1
                                                  1
                                                  1
Could not decode key with 6 flats and mode 255    1
Name: error, dtype: int64

In [170]:
songs.status.value_counts()

processed      456
unprocessed    209
Name: status, dtype: int64

In [171]:
songs[songs.status == 'processed']

,filename,source,song_name,midi_type,channels,n_channels,instruments,n_instruments,programs_mido,programs_pm,...,candidates,candidate_names,single_melody,multiple_melody,solo_channel,single_candidate,multi_channel,disagreement,status,error
1,./data/Complete Examples/Doug McKenzie/Alice I...,Doug McKenzie,Alice In Wonderland,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""do...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""do...",[dougmck@pipeline.com.au],False,False,False,True,False,False,processed,None
6,./data/Complete Examples/Doug McKenzie/Autumn ...,Doug McKenzie,Autumn Leaves,1.0,"{0, 1, 9}",3.0,"[Instrument(program=0, is_drum=False, name=""Pi...",4.0,"[(0, 0), (0, 1), (0, 9), (35, 1), (32, 9), (32...","[(0, False), (35, False), (32, True), (32, True)]",...,"[Instrument(program=0, is_drum=False, name=""Pi...",[Piano or Electric],False,False,False,True,False,True,processed,None
8,./data/Complete Examples/Doug McKenzie/Blue Ro...,Doug McKenzie,Blue Room,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Tr...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Tr...",[Track 3],False,False,False,True,False,False,processed,None
9,./data/Complete Examples/Doug McKenzie/But Bea...,Doug McKenzie,But Beautiful,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""So...",1.0,"[(0, 0), (0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""So...",[Solo piano],True,False,False,True,False,False,processed,None
10,./data/Complete Examples/Doug McKenzie/Come Ra...,Doug McKenzie,Come Rain Or Come Shine,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Ad...",2.0,"[(0, 0)]","[(0, False), (0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Ad...","[Added solo track, Accompaniment]",True,False,False,True,True,True,processed,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,./data/Complete Examples/Weimar DB/Woody Shaw ...,Weimar DB,Woody Shaw - Imagination,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Me...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Me...",[Melody],True,False,True,True,False,False,processed,None
661,./data/Complete Examples/Weimar DB/Wynton Mars...,Weimar DB,Wynton Marsalis - April In Paris,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name="""")]",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name="""")]",[],False,False,True,True,False,False,processed,None
662,./data/Complete Examples/Weimar DB/Zoot Sims -...,Weimar DB,Zoot Sims - All The Things You Are,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Me...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Me...",[Melody],True,False,True,True,False,False,processed,None
663,./data/Complete Examples/Weimar DB/Zoot Sims -...,Weimar DB,Zoot Sims - Night And Day,1.0,{0},1.0,"[Instrument(program=0, is_drum=False, name=""Me...",1.0,"[(0, 0)]","[(0, False)]",...,"[Instrument(program=0, is_drum=False, name=""Me...",[Melody],True,False,True,True,False,False,processed,None


In [172]:
unpro = songs[songs.status == 'unprocessed'].head(10)

In [173]:
songs.melody_track.isnull().sum()

5